In [1]:
!pip install torch==2.2
!pip install torchvision==0.17
!pip install matplotlib==3.5.2
!pip install scikit-image==0.19.3
!pip install pytorch-lightning==1.7.0
# !pip install torchmetrics==0.11.4

DEPRECATION: pytorch-lightning 1.7.0 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pytorch-lightning 1.7.0 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pytorch-lightning 1.7.0 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author

In [2]:
import os

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms
import pytorch_lightning as pl

In [3]:
class ConvNet(pl.LightningModule):

    def __init__(self):
        super(ConvNet, self).__init__()
        self.cn1 = nn.Conv2d(1, 16, 3, 1)
        self.cn2 = nn.Conv2d(16, 32, 3, 1)
        self.dp1 = nn.Dropout2d(0.10)
        self.dp2 = nn.Dropout2d(0.25)
        self.fc1 = nn.Linear(4608, 64) # 4608 is basically 12 X 12 X 32
        self.fc2 = nn.Linear(64, 10)
 
    def forward(self, x):
        x = self.cn1(x)
        x = F.relu(x)
        x = self.cn2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dp1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dp2(x)
        x = self.fc2(x)
        op = F.log_softmax(x, dim=1)
        return op

    def training_step(self, batch, batch_num):
        train_x, train_y = batch
        y_pred = self(train_x)
        training_loss = F.cross_entropy(y_pred, train_y)
        # optional
        self.log('train_loss', training_loss, on_epoch=True, prog_bar=True)
        return training_loss

    def validation_epoch_end(self, outputs):
        # optional
        avg_loss = torch.stack(outputs).mean()
        return avg_loss

    def test_step(self, batch, batch_num):
        # optional
        test_x, test_y = batch
        y_pred = self(test_x)
        test_loss = F.cross_entropy(y_pred, test_y)
        # optional
        self.log('test_loss', test_loss, on_step=True, on_epoch=True, prog_bar=True)
        return test_loss

    def test_epoch_end(self, outputs):
        # optional
        avg_loss = torch.stack(outputs).mean()
        return avg_loss

    def configure_optimizers(self):
        return torch.optim.Adadelta(self.parameters(), lr=0.5)

    def train_dataloader(self):
        return DataLoader(MNIST(os.getcwd(), train=True, download=True, 
                                transform=transforms.Compose([transforms.ToTensor(),
                                                              transforms.Normalize((0.1302,), (0.3069,))])), 
                                batch_size=32, num_workers=4)

    def val_dataloader(self):
        # optional
        return DataLoader(MNIST(os.getcwd(), train=True, download=True, 
                                transform=transforms.Compose([transforms.ToTensor(),
                                                              transforms.Normalize((0.1302,), (0.3069,))])), 
                                batch_size=32, num_workers=4)

    def test_dataloader(self):
        # optional
        return DataLoader(MNIST(os.getcwd(), train=False, download=True, 
                                transform=transforms.Compose([transforms.ToTensor(),
                                                              transforms.Normalize((0.1302,), (0.3069,))])), 
                                batch_size=32, num_workers=4)

In [4]:
model = ConvNet()

trainer = pl.Trainer(max_epochs=10)    
trainer.fit(model)   

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/Ashish.Jha/anaconda3/envs/python39/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1788: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(
/Users/Ashish.Jha/anaconda3/envs/python39/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:115: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")

  | Name | Type      | Params
-----------------------------------
0 | cn1  | Conv2d    | 160   
1 | cn2  | Conv2d    | 4.6 K 
2 | dp1  | Dropout2d | 0     
3 | dp2  | Dropout2d | 0     
4 | fc1  | Linear    | 294 K 
5 | fc2  | Linear    | 650   
-----------------------

Training: 0it [00:00, ?it/s]

/Users/Ashish.Jha/anaconda3/envs/python39/lib/python3.9/site-packages/torch/nn/functional.py:1347: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)
`Trainer.fit` stopped: `max_epochs=10` reached.


In [5]:
trainer.test()

/Users/Ashish.Jha/anaconda3/envs/python39/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1385: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
Restoring states from the checkpoint path at /Users/Ashish.Jha/code/personal/MasteringPyTorchV2/Chapter15/lightning_logs/version_14/checkpoints/epoch=9-step=18750.ckpt
Loaded model weights from checkpoint at /Users/Ashish.Jha/code/personal/MasteringPyTorchV2/Chapter15/lightning_logs/version_14/checkpoints/epoch=9-step=18750.ckpt


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_loss_epoch        0.03324906900525093
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss_epoch': 0.03324906900525093}]

In [6]:
# Start tensorboard.
%reload_ext tensorboard
%tensorboard --logdir lightning_logs/